In [1]:
!pip install ultralytics -q
!pip install roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 kB 2.7 MB/s eta 0:00:00


In [2]:
from roboflow import Roboflow
rf = Roboflow(api_key="3kx6YsZq5auR5mW8qIxJ")
project = rf.workspace("roboflow-jvuqo").project("football-players-detection-3zvbc")
version = project.version(1)
dataset = version.download("yolov5")
                

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to football-players-detection-1 in yolov5pytorch:: 100%|██████████| 1338/1338 [00:00<00:00, 3159.65it/s]


In [3]:
dataset.location


'/kaggle/working/football-players-detection-1'

In [4]:
import shutil

In [5]:
shutil.move("/kaggle/working/football-players-detection-1/valid","/kaggle/working/football-players-detection-1/football-players-detection-1/valid")
shutil.move("/kaggle/working/football-players-detection-1/test","/kaggle/working/football-players-detection-1/football-players-detection-1/test")
shutil.move("/kaggle/working/football-players-detection-1/train","/kaggle/working/football-players-detection-1/football-players-detection-1/train")

'/kaggle/working/football-players-detection-1/football-players-detection-1/train'

In [6]:
from ultralytics import YOLO


In [7]:

model = YOLO("/kaggle/input/yolov8m_13epochs/pytorch/default/1/best (2).pt")

results = model.predict("/kaggle/input/-dfl-bundesliga-460-mp4-videos-in-30sec-csv/DFL Bundesliga Data Shootout/train/A1606b0e6_0/A1606b0e6_0 (14).mp4",conf=0.1,save=True)

# print(results[0])
# print("****************************")
# for box in results[0].boxes:
#     print(box)




WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/749) /kaggle/input/-dfl-bundesliga-460-mp4-videos-in-30sec-csv/DFL Bundesliga Data Shootout/train/A1606b0e6_0/A1606b0e6_0 (14).mp4: 384x640 25 players, 96.5ms
video 1/1 (frame 2/749) /kaggle/input/-dfl-bundesliga-460-mp4-videos-in-30sec-csv/DFL Bundesliga Data Shootout/train/A1606b0e6_0/A1606b0e6_0 (14).mp4: 384x640 1 ball, 24 players, 13.5ms
video 1/1 (frame 3/749) /kaggle/input/-dfl-bundesliga-460-mp4-videos-in-30sec-csv/DFL Bundes

In [8]:
from IPython.display import Video

Video("/kaggle/working/runs/detect/predict/A1606b0e6_0 (14).avi")

In [9]:
import cv2

In [10]:
def read_video(video_path):
    capture = cv2.VideoCapture(video_path)
    frames = []
    while True:
        returned, frame = capture.read()
        if not returned:
            break
        frames.append(frame)
    return frames

In [11]:
from tqdm import tqdm

In [12]:
def save_video(out_video_frames,out_video_path):
    format_out = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(out_video_path,format_out,24,(out_video_frames[0].shape[1],out_video_frames[0].shape[0]))
    for frame in tqdm(out_video_frames):
        out.write(frame)
    out.release()

In [13]:
!pip install supervision -q

In [14]:
import supervision as sv

In [15]:
import numpy as np
import os
import pickle

In [16]:
detection_model = YOLO("/kaggle/input/trained-yolov5m-on-roboflow-annotated-data/best.pt")
tracker = sv.ByteTrack()

In [17]:
def detect_from_frames(frames):
    batch_size=20
    detections = []
    for i in range(0,len(frames),batch_size):
        batch_detections = detection_model.predict(frames[i:i+batch_size],conf=0.1)
        detections += batch_detections
    return detections

def get_object_tracks(frames,saved_tracks = False,saved_tracks_path=None):

    if(saved_tracks == True and saved_tracks_path != None and os.path.exists(saved_tracks_path)):
        with open(saved_tracks_path,"rb") as file:
            tracks = pickle.load(file)
        return tracks


    tracks={
        "players":[],
        "referees":[],
        "ball":[]
    }

    detections = detect_from_frames(frames)

    for frame_num, detection in enumerate(detections):
        class_names_dec = detection.names
        class_names_inv_dec = {v:k for k,v in class_names_dec.items()}

        detection_sv = sv.Detections.from_ultralytics(detection) 

        for subject_id,class_num in enumerate(detection_sv.class_id):
            if(class_names_dec[class_num] == "goalkeeper"):
                detection_sv.class_id[subject_id] = class_names_inv_dec["player"]

        detects_with_tracks = tracker.update_with_detections(detection_sv)

        tracks["players"].append({})
        tracks["referees"].append({})
        tracks["ball"].append({})

        for frame_detec in detects_with_tracks:
            bound_box = frame_detec[0].tolist()
            cls_id = frame_detec[3]
            track_id = frame_detec[4]

            if(cls_id == class_names_inv_dec["player"]):
                tracks['players'][frame_num][track_id] = {"bbox":bound_box}

            if(cls_id == class_names_inv_dec["referee"]):
                tracks['referees'][frame_num][track_id] = {"bbox":bound_box}

        for frame_detec in detection_sv:
            bound_box = frame_detec[0].tolist()
            cls_id = frame_detec[3]
            if(cls_id == class_names_inv_dec["ball"]):
                tracks['ball'][frame_num][track_id] = {"bbox":bound_box}

    if(saved_tracks_path!=None):
        with open(saved_tracks_path,"wb") as file:
            pickle.dump(tracks,file)



    return tracks

def draw_ellipse(frame,bbox,color,track_id,team=None):
    y2 = int(bbox[3])
    x = int((bbox[0]+bbox[2])/2)
    width = bbox[2]-bbox[0]
    cv2.ellipse(frame,center=(x,y2),axes=(int(width),int(width*0.25)),angle = 0.0,startAngle = -45,endAngle=235,color=color,thickness=2,lineType=cv2.LINE_4)


    rectangle_width = 40
    rectangle_height=20
    x1_rect = x - rectangle_width//2
    x2_rect = x + rectangle_width//2
    y1_rect = (y2- rectangle_height//2) +15
    y2_rect = (y2+ rectangle_height//2) +15

    if track_id is not None:
        cv2.rectangle(frame,
                      (int(x1_rect),int(y1_rect) ),
                      (int(x2_rect),int(y2_rect)),
                      color,
                      cv2.FILLED)

        x1_text = x1_rect+12
        if track_id > 99:
            x1_text -=10

        cv2.putText(
            frame,
            f"{track_id}",
            (int(x1_text),int(y1_rect+15)),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.6,
            (0,0,0),
            2
        )
        if(team!=None):
            cv2.putText(
                frame,
                f"team: {team}",
                (int(x1_text),int(y1_rect+35)),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                (0,0,0),
                2
            )

    return frame

def draw_triangle(frame,bbox,color):
    y = int(bbox[1])
    x = int((bbox[2]+bbox[0])/2)

    triangle_pts = np.array([
        [x,y],
        [x-10,y-20],
        [x+10,y-20]
    ])

    cv2.drawContours(frame,[triangle_pts],0,color,cv2.FILLED)
    cv2.drawContours(frame,[triangle_pts],0,(0,0,0),2)

    return frame

def draw_annotations(video_frames,tracks):
    out_video_frames=[]
    for frame_num,frame in enumerate(video_frames):
        frame= frame.copy()
        player_dict = tracks["players"][frame_num]
        ref_dict = tracks["referees"][frame_num]
        ball_dict = tracks["ball"][frame_num]

        for track_id,player in player_dict.items():
            team_color = player.get("team_color",(255,255,255))
            team = player["team"]
            frame = draw_ellipse(frame,player["bbox"],team_color,track_id,team)

        for track_id,ref in ref_dict.items():
            frame = draw_ellipse(frame,player["bbox"],(0,0,0),track_id)

        for track_id,ball in ball_dict.items():
            frame = draw_triangle(frame,ball["bbox"],(0,255,0))

        out_video_frames.append(frame)
    return out_video_frames


In [18]:
from sklearn.cluster import KMeans

In [19]:
import matplotlib.pyplot as plt
%matplotlib inline


In [28]:
# class Team_assigner:
#     def __init__(self):
#         self.team_colors = {}
#         self.player_team_dic = {}
#         pass
    
# #     def get_player_color(self,frame,bound_box):
# #         image = frame[int(bound_box[1]):int((bound_box[3]+bound_box[1])/2),int(bound_box[0]):int(bound_box[2])]
# #         gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
# #         # Flatten the image to reshape for KMeans
# #         img_tmp = gray_image.reshape(-1, 1)
# #         kmeans = KMeans(n_clusters=2,init="k-means++",n_init=10)
# #         kmeans.fit(img_tmp)
        
# #         labels = kmeans.labels_
        
# #         segmented_image = labels.reshape(image.shape[0],image.shape[1])
# #         corner_points = [segmented_image[0,0],segmented_image[0,-1],segmented_image[-1,0],segmented_image[-1,-1]]
# #         bkground = max(set(corner_points),key=corner_points.count)
# #         player_color = kmeans.cluster_centers_[1-bkground]
# #         return player_color
#     def get_player_color(self, frame, bound_box):
#         # Crop the bounding box area of the player from the frame
#         image = frame[int(bound_box[1]):int((bound_box[3] + bound_box[1]) / 2), int(bound_box[0]):int(bound_box[2])]

#         # Apply Gaussian blur to reduce noise and smooth the image
#         blurred_image = cv2.GaussianBlur(image, (5, 5), 0)

#         # Convert the image from BGR to RGB
#         rgb_image = cv2.cvtColor(blurred_image, cv2.COLOR_BGR2RGB)

#         # Reshape the image for KMeans (3 channels: R, G, B)
#         img_tmp = rgb_image.reshape(-1, 3)

#         # Use KMeans to cluster into 2 color groups (player vs background)
#         kmeans = KMeans(n_clusters=2, init="k-means++", n_init=10)
#         kmeans.fit(img_tmp)

#         labels = kmeans.labels_

#         # Reshape the labels back into the image's shape
#         segmented_image = labels.reshape(rgb_image.shape[0], rgb_image.shape[1])

#         # Sample the central pixels to focus on the player's uniform
#         center_segment = segmented_image[rgb_image.shape[0] // 4 : 3 * rgb_image.shape[0] // 4,
#                                          rgb_image.shape[1] // 4 : 3 * rgb_image.shape[1] // 4]

#         # Determine the dominant cluster in the central area
#         unique, counts = np.unique(center_segment, return_counts=True)
#         player_cluster = unique[np.argmax(counts)]

#         # Get the average color for the player's cluster (use cluster centers)
#         player_color = kmeans.cluster_centers_[player_cluster]

#         return player_color

        
#     def assigne_team_colors(self,frame,player_detections):
#         player_colors = []
#         for _,player_detection in player_detections.items():
#             bbox = player_detection["bbox"]
#             player_color =  self.get_player_color(frame,bbox)
#             player_colors.append(player_color)
            
#         kmeans = KMeans(n_clusters=2,init="k-means++",n_init=10)
#         kmeans.fit(player_colors)
        
#         self.kmeans = kmeans
        
#         self.team_colors[1] = kmeans.cluster_centers_[0]
#         self.team_colors[2] = kmeans.cluster_centers_[1] 
        
    
#     def get_player_team(self,frame,player_bbox,player_id):
# #         if player_id in self.player_team_dic:
# #             return self.player_team_dic[player_id]
        
#         player_color= self.get_player_color(frame,player_bbox)
# #         print("color of player ",player_id," = ",player_color.reshape(1,-1))
#         team_id = self.kmeans.predict(player_color.reshape(1,-1))[0]
#         team_id +=1
        
#         return team_id
# from sklearn.neighbors import KNeighborsClassifier
# import numpy as np

# class Team_assigner:
#     def __init__(self):
#         self.team_colors = []  # Will hold the training data (player colors)
#         self.team_labels = []  # Will hold the corresponding team IDs (labels)
#         self.knn = KNeighborsClassifier(n_neighbors=5)  # Initialize KNN classifier with 3 neighbors
#         self.player_team_dic = {}  # To store team assignment for each player
#         self.is_trained = False  # To check if KNN is trained

#     def get_player_color(self, frame, bound_box):
#         # Extract the player uniform color from the bounding box (same logic as before)
#         image = frame[int(bound_box[1]):int((bound_box[3]+bound_box[1])/2), int(bound_box[0]):int(bound_box[2])]
#         gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
#         # Flatten the image to reshape for KMeans
#         img_tmp = gray_image.reshape(-1, 1)
#         kmeans = KMeans(n_clusters=2, init="k-means++", n_init=1)
#         kmeans.fit(img_tmp)
        
#         labels = kmeans.labels_
#         segmented_image = labels.reshape(image.shape[0], image.shape[1])
#         corner_points = [segmented_image[0,0], segmented_image[0,-1], segmented_image[-1,0], segmented_image[-1,-1]]
#         bkground = max(set(corner_points), key=corner_points.count)
#         player_color = kmeans.cluster_centers_[1 - bkground]
        
#         return player_color
    
#     def assigne_team_colors(self, frame, player_detections):
#         player_colors = []

#         # Iterate through player detections and extract their colors
#         for player_id, player_detection in player_detections.items():
#             bbox = player_detection["bbox"]
#             player_color = self.get_player_color(frame, bbox)
#             player_colors.append(player_color)

#         # Convert the list of player colors to a numpy array
#         player_colors = np.array(player_colors)

#         # Use K-means to cluster the player colors into two teams (n_clusters=2)
#         kmeans = KMeans(n_clusters=2, init="k-means++", n_init=10)
#         kmeans.fit(player_colors)
#         print(player_colors)
#         # The K-means labels will serve as team labels (either 0 or 1 for each team)
#         team_labels = kmeans.labels_  # These are the labels for each player's team
#         print(team_labels)
#         # Store the player colors and their team labels
#         self.team_colors = player_colors
#         self.team_labels = team_labels

#         # Train the KNN model using the player colors and their team labels
#         self.knn.fit(self.team_colors, self.team_labels)
#         self.is_trained = True

    
#     def get_player_team(self, frame, player_bbox, player_id):
#         if player_id in self.player_team_dic:
#             return self.player_team_dic[player_id]
        
#         # Extract player color and predict the team using KNN
#         player_color = self.get_player_color(frame, player_bbox)
        
#         if not self.is_trained:
#             raise ValueError("The KNN model has not been trained. Call 'assign_team_colors' first.")
        
#         # Predict the team based on the nearest neighbors
#         team_id = self.knn.predict(player_color.reshape(1, -1))[0]
        
#         # Cache the team assignment for this player
#         self.player_team_dic[player_id] = team_id
        
#         return team_id
import cv2
import numpy as np
from sklearn.cluster import KMeans

class Team_assigner:
    def __init__(self):
        self.team_colors = {}  # Stores team colors assigned after KMeans
        self.player_team_dic = {}  # Cache of player team assignments
        self.kmeans = None  # KMeans model used to predict teams

    def get_player_color(self, frame, bound_box):
        """
        Extract the dominant color from a player's bounding box.
        """
        # Crop the player region from the bounding box
        image = frame[int(bound_box[1]):int((bound_box[3] + bound_box[1]) / 2), int(bound_box[0]):int(bound_box[2])]

        # Convert the image from BGR to HSV to capture hue information better
        hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

        # Apply Gaussian blur to reduce noise and smooth the image
        blurred_image = cv2.GaussianBlur(hsv_image, (5, 5), 0)

        # Reshape the image for KMeans (we're clustering based on Hue only)
        img_tmp = blurred_image[:, :, 0].reshape(-1, 1)  # Use only the Hue channel

        # Use KMeans to cluster into 2 color groups (player uniform vs background)
        kmeans = KMeans(n_clusters=2, init="k-means++", n_init=10)
        kmeans.fit(img_tmp)

        # Get the labels for each pixel in the image
        labels = kmeans.labels_

        # Reshape the labels back into the image's shape
        segmented_image = labels.reshape(image.shape[0], image.shape[1])

        # Focus on the central part of the uniform to avoid borders and background
        center_segment = segmented_image[image.shape[0] // 4: 3 * image.shape[0] // 4,
                                         image.shape[1] // 4: 3 * image.shape[1] // 4]

        # Determine the dominant cluster in the central area (either 0 or 1)
        unique, counts = np.unique(center_segment, return_counts=True)
        player_cluster = unique[np.argmax(counts)]

        # Get the average color for the player's cluster (use cluster centers)
        player_color = kmeans.cluster_centers_[player_cluster]

        return player_color

    def assigne_team_colors(self, frame, player_detections):
        """
        Assign team colors using KMeans on all detected players' colors.
        """
        player_colors = []
        for _, player_detection in player_detections.items():
            bbox = player_detection["bbox"]
            player_color = self.get_player_color(frame, bbox)
            player_colors.append(player_color)

        # Convert list of colors into a numpy array for KMeans clustering
        player_colors = np.array(player_colors)

        # Use KMeans to classify players into 2 teams based on their dominant uniform color
        kmeans = KMeans(n_clusters=2, init="k-means++", n_init=10)
        kmeans.fit(player_colors)

        # Store the KMeans model for later predictions
        self.kmeans = kmeans

        # Assign the team colors to each cluster
        self.team_colors[1] = kmeans.cluster_centers_[0]
        self.team_colors[2] = kmeans.cluster_centers_[1]

    def get_player_team(self, frame, player_bbox, player_id):
        """
        Predict the player's team based on their uniform color.
        """
        if player_id in self.player_team_dic:
            return self.player_team_dic[player_id]  # Return cached team assignment

        # Extract the player's color
        player_color = self.get_player_color(frame, player_bbox)

        if self.kmeans is None:
            raise ValueError("Team colors have not been assigned. Call 'assigne_team_colors' first.")

        # Predict the team based on the closest cluster
        team_id = self.kmeans.predict(player_color.reshape(1, -1))[0]
        team_id += 1  # Teams are 1-indexed (i.e., team 1, team 2)

        # Cache the player's team assignment for future reference
        self.player_team_dic[player_id] = team_id

        return team_id

    def clear_cache(self):
        """
        Clear the player-team assignment cache (e.g., when a new game starts).
        """
        self.player_team_dic.clear()


In [31]:
video_frames = read_video("/kaggle/input/-dfl-bundesliga-460-mp4-videos-in-30sec-csv/DFL Bundesliga Data Shootout/train/A1606b0e6_0/A1606b0e6_0 (14).mp4")
# tracker = Tracker("/kaggle/input/trained-yolov5m-on-roboflow-annotated-data/best.pt")

video_tracks = get_object_tracks(frames=video_frames,saved_tracks=True,saved_tracks_path="/kaggle/working/saved_tracks.pkl")
print("tracks present")


team_assigner = Team_assigner()

team_assigner.assigne_team_colors(video_frames[0],video_tracks["players"][0])
print("team assigner done")
    
for frame_num,player_tracks in enumerate(video_tracks["players"]):
    team_assigner.clear_cache()
    for player_id,track in player_tracks.items():
        team = team_assigner.get_player_team(video_frames[frame_num],track["bbox"],player_id)
        video_tracks["players"][frame_num][player_id]["team"]=team
        video_tracks["players"][frame_num][player_id]["team_color"]=team_assigner.team_colors[team]
        

out_video_frames =draw_annotations(video_frames,video_tracks)


save_video(out_video_frames,"/kaggle/working/output_vid55.avi")

tracks present
team assigner done


100%|██████████| 749/749 [00:10<00:00, 69.81it/s]
